In [2]:
import numpy as np
with open("scan_001.xyz","r") as fid:
    pointcloud_01 = fid.read()

pointcloud_01 = np.array([[float(p) for p in pc.split()] for pc in pointcloud_01.strip("\n").split("\n")]).astype(np.float32)

with open("scan_002.xyz","r") as fid:
    pointcloud_02 = fid.read()
    
pointcloud_02 = np.array([[float(p) for p in pc.split()] for pc in pointcloud_02.strip("\n").split("\n")]).astype(np.float32)

In [3]:
import torch
print(torch.__version__)

1.12.1


In [4]:
occ_01 = torch.load('../output/pred_001_voted.pt')

In [5]:
idx = occ_01.permute((1, 2, 0)).nonzero() + torch.Tensor([[0.5, 0.5, 0.5]])# xyz

In [6]:
pred_01 = (idx*torch.Tensor([[0.49479, 0.49479, 0.3125]])).type(torch.float32)

In [22]:
pred_01.shape

torch.Size([1587578, 3])

(38402, 3)

(55413, 3)

AttributeError: 'numpy.ndarray' object has no attribute 'type'

In [14]:
from kaolin.metrics.pointcloud import f_score
print(f_score(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pred_01[np.newaxis, :, :]).to('cuda:0'), radius=0.5, eps=1e-08))

<ipython-input-14-d18a8bee9a77>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print(f_score(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pred_01[np.newaxis, :, :]).to('cuda:0'), radius=0.5, eps=1e-08))


tensor([0.0059], device='cuda:0')


In [45]:
print(f_score(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pred_01[np.newaxis, :, :]).to('cuda:0'), radius=1, eps=1e-08))

<ipython-input-45-f4ea434f018b>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print(f_score(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pred_01[np.newaxis, :, :]).to('cuda:0'), radius=1, eps=1e-08))


tensor([0.0271], device='cuda:0')


In [15]:
print(f_score(torch.tensor(pred_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), radius=1, eps=1e-08))

<ipython-input-15-917f0c4f2e80>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print(f_score(torch.tensor(pred_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), radius=1, eps=1e-08))


tensor([0.0019], device='cuda:0')


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Feb__7_19:32:13_PST_2023
Cuda compilation tools, release 12.1, V12.1.66
Build cuda_12.1.r12.1/compiler.32415258_0


In [ ]:
!pip install --upgrade jinja2
!pip install -q kaolin==0.13.0 -f https://nvidia-kaolin.s3.us-east-2.amazonaws.com/torch-1.13.1_cu116.html

##F-1 Score
See F-1 score definition for pointcloud at Nvidia's KAOLIN documentation:
https://kaolin.readthedocs.io/en/latest/modules/kaolin.metrics.pointcloud.html
Using 0.1, 0.5, 1.0 for radius

In [ ]:
from kaolin.metrics.pointcloud import f_score
print(f_score(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), radius=0.1, eps=1e-08))
print(f_score(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pointcloud_02[np.newaxis, :, :]).to('cuda:0'), radius=0.1, eps=1e-08))
print(f_score(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pointcloud_02[np.newaxis, :, :]).to('cuda:0'), radius=0.5, eps=1e-08))
print(f_score(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pointcloud_02[np.newaxis, :, :]).to('cuda:0'), radius=1.0, eps=1e-08))

tensor([1.0000], device='cuda:0', dtype=torch.float64)
tensor([0.0001], device='cuda:0', dtype=torch.float64)
tensor([0.0177], device='cuda:0', dtype=torch.float64)
tensor([0.0402], device='cuda:0', dtype=torch.float64)


## Chamfer Distance
See Chamfer definition for pointcloud at Nvidia's KAOLIN documentation: 
https://kaolin.readthedocs.io/en/latest/modules/kaolin.metrics.pointcloud.html
We use default parameters for symmetric distance

In [16]:
from kaolin.metrics.pointcloud import chamfer_distance
print(chamfer_distance(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0')))
print(chamfer_distance(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pointcloud_02[np.newaxis, :, :]).to('cuda:0')))

tensor([0.], device='cuda:0')
tensor([1121.5381], device='cuda:0')


In [17]:
print(chamfer_distance(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pred_01[np.newaxis, :, :]).to('cuda:0')))

<ipython-input-17-2ea9da51ed73>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print(chamfer_distance(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pred_01[np.newaxis, :, :]).to('cuda:0')))


tensor([1418.4249], device='cuda:0')


## Direct Hausdorff
See Scipy's documentation for direct Hausdorff distance:
https://scipy.github.io/devdocs/reference/generated/scipy.spatial.distance.directed_hausdorff.html

In [18]:
from scipy.spatial.distance import directed_hausdorff
d, _, _ = directed_hausdorff(pointcloud_01, pointcloud_01)
print("Hausdorff distance between PC1 and itself", d)
d, _, _ = directed_hausdorff(pointcloud_01, pointcloud_02)
print("Hausdorff distance between PC1 and PC2", d)

Hausdorff distance between PC1 and itself 0.0
Hausdorff distance between PC1 and PC2 84.35377426860708


In [19]:
d, _, _ = directed_hausdorff(pointcloud_01, pred_01)
print("Hausdorff distance between PC1 and PC2", d)

Hausdorff distance between PC1 and PC2 50.87637815448629


## Mean Surface Distance / Average Symmetric Surface Distance
See the definition here:
https://github.com/emrekavur/CHAOS-evaluation/blob/master/CHAOS_Metrics_and_Evaluation_Method.pdf

In [20]:
import numpy as np
from kaolin.metrics.pointcloud import sided_distance

def mean_surface_distance(p1: torch.tensor, p2: torch.tensor) -> float:
  msd = torch.sum(sided_distance(p1, p2)[0]).detach().cpu().numpy() + torch.sum(sided_distance(p2, p1)[0]).detach().cpu().numpy()
  msd /= p1.size()[1] + p2.size()[1]
  return msd

print(mean_surface_distance(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0')))
print(mean_surface_distance(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pointcloud_02[np.newaxis, :, :]).to('cuda:0')))

0.0
530.9226882694664


In [21]:
print(mean_surface_distance(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pred_01[np.newaxis, :, :]).to('cuda:0')))

<ipython-input-21-3afd9555236e>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print(mean_surface_distance(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pred_01[np.newaxis, :, :]).to('cuda:0')))


808.2735777807845


## Residual Mean Square Distance
See definition here: https://github.com/emrekavur/CHAOS-evaluation/blob/master/CHAOS_Metrics_and_Evaluation_Method.pdf

In [7]:
import numpy as np
from kaolin.metrics.pointcloud import sided_distance

def mean_surface_distance(p1: torch.tensor, p2: torch.tensor) -> float:
  msd = torch.sum(torch.square(sided_distance(p1, p2)[0])).detach().cpu().numpy() + torch.sum(torch.square(sided_distance(p2, p1)[0])).detach().cpu().numpy()
  msd /= p1.size()[1] + p2.size()[1]
  return msd

print(mean_surface_distance(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0')))
print(mean_surface_distance(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pointcloud_02[np.newaxis, :, :]).to('cuda:0')))

0.0
860766.9668176731


In [8]:
print(mean_surface_distance(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pred_01[np.newaxis, :, :]).to('cuda:0')))

<ipython-input-8-3afd9555236e>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  print(mean_surface_distance(torch.tensor(pointcloud_01[np.newaxis, :, :]).to('cuda:0'), torch.tensor(pred_01[np.newaxis, :, :]).to('cuda:0')))


1233698.694881856
